In [ ]:
import os
import sys
import json
sys.path.append("../../../")
current_file_dir = os.getcwd()


from prompt_scope.core.llms.dashscope_llm import DashscopeLLM
from prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer import StaticTipsOptimizer
from prompt_scope.core.llms.vllm_llm import VLLMHandler

# Step1: Load data

In [2]:
with open("../../data/college_medical_statistics/sample_train_data.jsonl", "r") as f:
    train_set = []
    for line in f:
        train_set.append(json.loads(line))

with open("../../data/college_medical_statistics/sample_test_data.jsonl", "r") as f:
    test_set = []
    for line in f:
        test_set.append(json.loads(line))

with open("../../data/college_medical_statistics/init_system_prompt.txt", "r") as f:
    init_system_prompt = f.read().strip()

# Step2: Define a scoring function

In [3]:
def is_good_case(prediction, ground_truth):
    prediction = prediction.split("<answer>")[-1].split("</answer>")[0].strip()
    ground_truth = ground_truth
    if ground_truth in prediction:
        return True, 1 # Good case, score
    else:
        return False, 0 # Bad case, score

# Step3: Run optimizer

In [ ]:
# Use VLLM (both infer_llm and optim_llm are Qwen2-72B-Instruct, \
# tensor_parallel_size and device may need to be adjusted according to the model and device)

# infer_llm = VLLMHandler(
#         model_path="PATH_TO_MODEL",
#         temperature=0.0,
#         tensor_parallel_size=4,
#         stop_tokens=["<|eot_id|>","<|start_header_id|>"],
#         device="cuda:0"
#     )
# optim_llm = VLLMHandler(
#         model_path="PATH_TO_MODEL",
#         temperature=0.0,
#         tensor_parallel_size=4,
#         stop_tokens=["<|eot_id|>","<|start_header_id|>"],
#         device="cuda:4"
#     )

# Use Dashscope
infer_llm = DashscopeLLM(model="qwen-plus", temperature=0.0)
optim_llm = DashscopeLLM(model="qwen-max", temperature=0.0)

In [ ]:
stg = StaticTipsOptimizer(
    init_system_prompt=init_system_prompt,
    infer_llm=infer_llm,
    optim_llm=optim_llm,
    train_set=train_set,
    test_set=test_set,
    is_good_case_func=is_good_case,
    details_save_dir=f"{current_file_dir}/details_result",
    save_steps = 30,
    epoch = 5
)
optimized_prompt = stg.run()

2025-02-07 16:42:48.569 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:_before_run:77 - 
============== evaluating initial system prompt on test ===============
Score: 100%|██████████████████████████████████| 53/53 [00:00<00:00, 3437.32it/s]
2025-02-07 16:43:28.637 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:_before_run:84 - test_avg_score: 
0.7547169811320755
Epoch-1/10 Training:  55%|████████████          | 29/53 [05:31<02:46,  6.92s/it]2025-02-07 16:49:09.873 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:121 - 
=====Epoch-1 Step-30/530=====
2025-02-07 16:49:09.874 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:122 - 
=====Epoch-1/10 Step-30/530 updated tips=====
- 当已知相关系数 \(r\) 有统计学意义时，建议考虑 \(b\) 也可能具有统计学意义，因为两者反映了同一线性关系的不同方面。
- 不建议仅凭 \(r\) 的显著性就完全排除 \(b\) 可能不具备统计学意义的可能性，但在没有额外信息的情况下，可以合理推测 \(b\) 同样显著。

- 当面对显著性检验结果为不拒绝\(H_0\)时，建议深入探讨数据特征及背景信息，考虑是否存

Predict: 100%|██████████████████████████████████| 53/53 [00:16<00:00,  3.23it/s]

Score: 100%|██████████████████████████████████| 53/53 [00:00<00:00, 3031.35it/s]
2025-02-07 16:51:49.112 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:135 - 
test_avg_score: 0.8113207547169812
Epoch-2/10 Training:  68%|██████████████▉       | 36/53 [04:33<00:44,  2.60s/it]2025-02-07 16:55:43.879 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:121 - 
=====Epoch-2 Step-90/530=====
2025-02-07 16:55:43.881 | DEBUG    | prompt_scope.core.optimizer.tips_optimizer.static_tips_optimizer:run:122 - 
=====Epoch-2/10 Step-90/530 updated tips=====
- 当已知相关系数 \(r\) 有统计学意义时，建议考虑 \(b\) 也可能具有统计学意义，因为两者反映了同一线性关系的不同方面。
- 不建议仅凭 \(r\) 的显著性就完全排除 \(b\) 可能不具备统计学意义的可能性，但在没有额外信息的情况下，可以合理推测 \(b\) 同样显著。

- 当面对显著性检验结果为不拒绝\(H_0\)时，建议深入探讨数据特征及背景信息，考虑是否存在非线性关系或其他形式的相关性，不建议立即否定模型的所有潜在价值。
- 在评估模型应用价值时，建议综合考量统计显著性之外的因素，如业务需求、数据质量等，不建议单凭统计测试结果做出最终判断。

- 当考虑使用近似正态\(u\)检验时，建

In [ ]:
print(optimized_prompt)